In [3]:
DK-SAP

In [ ]:
import hashlib
from py_ecc.secp256k1 import *
import sha3
from eth_account import Account
from ecdsa import SigningKey, SECP256k1 
import time

In [4]:
def sk_PK_HPK_WA(sk):
    # PK = secp256k1.privtopub(s.to_bytes(32, "big"))
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))
    
    HPK = sha3.keccak_256(PK[0].to_bytes(32, "big")+PK[1].to_bytes(32, "big")).hexdigest()
    WA = "0x"+ HPK[-40:]
    return PK, HPK, WA
    
sk_george = int(0x13803677559c049a98a1576ad2f1fd15bdd9f81886f225c0850e883bd7d4cf63)
PK_george, HPK_george, WA_george = sk_PK_HPK_WA(sk_george)

name = 'george'
print(name)
print('==== G: \n', secp256k1.G)
print('==== sk: \n', sk_george)
print('==== PK=G*sk \n', PK_george)
print('==== HPK=h256(PK) \n', HPK_george)
print('==== WA=-40(HPK): \n', WA_george)


def get_PK(sk):
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))
    return PK

def sa_PK_scan_spend(sk_scan, sk_spend):
    PK_scan, HPK_scan, WA_scan = sk_PK_HPK_WA(sk_scan)
    PK_spend, HPK_spend, WA_spend = sk_PK_HPK_WA(sk_spend)
    return PK_scan, PK_spend

def register_PKscan_PKspend(PK_scan, PK_spend):
    pass

def gen_SA(r, PK_scan, PK_spend):
    R = get_PK(r)
    
    ss_alice = secp256k1.multiply(PK_scan, r)
    #print('Alice gen ss: \n', ss_alice)

    hss_alice = sha3.keccak_256(ss_alice[0].to_bytes(32, "big") + ss_alice[1].to_bytes(32, "big")).hexdigest()
    hss_alice = bytearray.fromhex(hss_alice)
    #print('Alice gen h256(ss): \n', hss_alice)

    PK_sa_alice = secp256k1.add(PK_spend, secp256k1.privtopub(hss_alice))
    #print('Alice gen PK_sa: \n', PK_sa_alice)

    HPK_sa_alice = sha3.keccak_256(PK_sa_alice[0].to_bytes(32, "big")
                        +PK_sa_alice[1].to_bytes(32, "big")
                        ).hexdigest()
    #print('Alice gen HPK_sa: \n', HPK_sa_alice)

    WA_sa_alice = "0x"+ HPK_sa_alice[-40:]
    #print('Alice gen WA_sa: \n', WA_sa_alice)
    
    return R, WA_sa_alice

def scan_SA(R, sk_scan, PK_spend):
    ss_bob = secp256k1.multiply(R, sk_scan)

    hss_bob = sha3.keccak_256(ss_bob[0].to_bytes(32, "big") + ss_bob[1].to_bytes(32, "big")).hexdigest()
    hss_bob = bytearray.fromhex(hss_bob)
    #print('Bob gen h256(ss): \n', hss_bob)
    
    PK_sa_bob = secp256k1.add(PK_spend, secp256k1.privtopub(hss_bob))
    #print('Alice gen PK_sa: \n', PK_sa_alice)

    HPK_sa_bob = sha3.keccak_256(PK_sa_bob[0].to_bytes(32, "big")
                            +PK_sa_bob[1].to_bytes(32, "big")
                            ).hexdigest()
    #print('Bob gen HPK_sa: \n', HPK_sa_alice)

    WA_sa_bob = "0x"+ HPK_sa_bob[-40:]
    #print('Bob gen WA_sa: \n', WA_sa_bob)
    
    return WA_sa_bob

def spend_SA(R, sk_scan, sk_spend):
    ss_bob = secp256k1.multiply(R, sk_scan)

    Hss_bob = sha3.keccak_256(ss_bob[0].to_bytes(32, "big") + ss_bob[1].to_bytes(32, "big")).hexdigest()
    #print('Bob gen h256(ss): \n', Hss_bob)

    sk_sa_bob = sk_spend + int(Hss_bob, 16)
    #print("Bob gen sk_sa: \n", sk_sa_bob)

    PK_sa_bob = secp256k1.privtopub(sk_sa_bob.to_bytes(32, "big"))
    #print('Bob gen PK_sa: \n', PK_sa_bob)

    HPK_sa_bob = sha3.keccak_256(PK_sa_bob[0].to_bytes(32, "big")
                            +PK_sa_bob[1].to_bytes(32, "big")
                            ).hexdigest()
    #print('Bob gen HPK_sa: \n', HPK_sa_bob)

    WA_sa_bob = "0x"+ HPK_sa_bob[-40:]
    #print('Bob gen WA_sa: \n', WA_sa_bob)
    
    return sk_sa_bob, WA_sa_bob

    
def register_R(R):
    pass

def send_to_SA(WA_SA):
    pass

george
==== G: 
 (55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)
==== sk: 
 8820476458925801522381692954830752984895658962939311888421485220456817807203
==== PK=G*sk 
 (66308977532324777246755440929893538803702139848914800508388379439121578598121, 74282901316977640338205813941139843601962206750695816950367302342758470583341)
==== HPK=h256(PK) 
 6a5904172647e8370bf61e308a31e51ad0cf970cfb4de3c7c9929a0813fbfea1
==== WA=-40(HPK): 
 0x8a31e51ad0cf970cfb4de3c7c9929a0813fbfea1


In [25]:
for i in range(100):
    sum = 0
    start = time.perf_counter()
    sk_scan = int(0x0000000000000000000000000000000000000000000000000000000000000002) # private key
    sk_spend = int(0x0000000000000000000000000000000000000000000000000000000000000003) # private key
    PK_scan, PK_spend = sa_PK_scan_spend(sk_scan, sk_spend)
    register_PKscan_PKspend(PK_scan, PK_spend)
    r = int(0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30) # private key
    R, SA_alice = gen_SA(r, PK_scan, PK_spend)
    register_R(R)
    send_to_SA(SA_alice)
    SA_scan = scan_SA(R, sk_scan, PK_spend)
    SA_scan.lower() == SA_alice.lower()
    sk_SA_spend, SA_spend = spend_SA(R, sk_scan, sk_spend)
    SA_to_spend = Account.from_key((sk_SA_spend).to_bytes(32, "big")).address
    SA_to_spend.upper() == SA_spend.upper()
    end = time.perf_counter()
    elapsed_time = end - start
    int(elapsed_time)
    print(elapsed_time)
    i+1


0.021895808999943256
0.020113136999952985
0.020734647999688605
0.01941215499982718
0.01840258400034145
0.01837149999983012
0.019438518000242766
0.01823041200032094
0.018894569000167394
0.01779811400001563
0.018601277000016125
0.01781421900022906
0.017842034999830503
0.019972980999682477
0.021955639999760024
0.02061147099993832
0.020103144999666256
0.021313420999831578
0.019876134999776696
0.020181369000056293
0.02028160600002593
0.01973272299983364
0.019392066999898816
0.020256251999853703
0.01958189000015409
0.020701372000075935
0.018615782000324543
0.018570950999674096
0.018056378999972367
0.01763648100040882
0.017614713000057236
0.017513546999907703
0.018353231999753916
0.017676941999980045
0.01795992600000318
0.017778200000066136
0.017657954999776848
0.019069780999871
0.02045440800020515
0.018695258000207104
0.018136518999654072
0.018854182000268338
0.018764114000077825
0.01875601900019319
0.01902629699998215
0.02030032599986953
0.020153610999841476
0.01987511600009384
0.0208044990

In [ ]:
DK-SAP Paillier

In [21]:
import hashlib
from py_ecc.secp256k1 import *
import sha3
from eth_account import Account
from ecdsa import SigningKey, SECP256k1 
from phe import paillier
import time
import timeit

In [22]:
def sk_PK_HPK_WA(sk):
    # PK = secp256k1.privtopub(s.to_bytes(32, "big"))
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))  
    HPK = sha3.keccak_256(PK[0].to_bytes(32, "big")+PK[1].to_bytes(32, "big")).hexdigest()
    WA = "0x"+ HPK[-40:]
    return PK, HPK, WA
    
sk_bob = int(0x13803677559c049a98a1576ad2f1fd15bdd9f81886f225c0850e883bd7d4cf63)
PK_bob, HPK_bob, WA_bob = sk_PK_HPK_WA(sk_bob)

name = 'Bob'
print(name)
#print('==== G: \n', secp256k1.G)
print('==== sk_bob: \n', sk_bob)
print('==== PK_bob=G*sk_bob \n', PK_bob)
#print('==== HPK=h256(PK) \n', HPK_bob)
print('==== WA2=-40(HPK): \n', WA_bob)


def get_PK(sk):
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))
    return PK

def sk_PK_HPK_WA(sk):
    # PK = secp256k1.privtopub(s.to_bytes(32, "big"))
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))  
    HPK = sha3.keccak_256(PK[0].to_bytes(32, "big")+PK[1].to_bytes(32, "big")).hexdigest()
    WA = "0x"+ HPK[-40:]
    return PK, HPK, WA
    
sk_bob = int(0x13803677559c049a98a1576ad2f1fd15bdd9f81886f225c0850e883bd7d4cf63)
PK_bob, HPK_bob, WA_bob = sk_PK_HPK_WA(sk_bob)

name = 'Bob'
print(name)
#print('==== G: \n', secp256k1.G)
print('==== sk_bob: \n', sk_bob)
print('==== PK_bob=G*sk_bob \n', PK_bob)
#print('==== HPK=h256(PK) \n', HPK_bob)
print('==== WA2=-40(HPK): \n', WA_bob)


def get_PK(sk):
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))
    return PK

Bob
==== sk_bob: 
 8820476458925801522381692954830752984895658962939311888421485220456817807203
==== PK_bob=G*sk_bob 
 (66308977532324777246755440929893538803702139848914800508388379439121578598121, 74282901316977640338205813941139843601962206750695816950367302342758470583341)
==== WA2=-40(HPK): 
 0x8a31e51ad0cf970cfb4de3c7c9929a0813fbfea1
Bob
==== sk_bob: 
 8820476458925801522381692954830752984895658962939311888421485220456817807203
==== PK_bob=G*sk_bob 
 (66308977532324777246755440929893538803702139848914800508388379439121578598121, 74282901316977640338205813941139843601962206750695816950367302342758470583341)
==== WA2=-40(HPK): 
 0x8a31e51ad0cf970cfb4de3c7c9929a0813fbfea1


In [23]:
for i in range(100):
    sum = 0
    start = time.perf_counter()
    public_key, private_key = paillier.generate_paillier_keypair()
    C2 = public_key.encrypt(sk_bob)
    sk_alice_fhe = int(0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30)
    PK_alice_fhe = get_PK(sk_alice_fhe)
    PK_fhe_alice = secp256k1.add(PK_alice_fhe, PK_bob)
    HPK_fhe_alice = sha3.keccak_256(PK_fhe_alice[0].to_bytes(32,'big')+PK_fhe_alice[1].to_bytes(32,'big')).hexdigest()
    WA_fhe_alice = "0x"+ HPK_fhe_alice[-40:]
    C1 = public_key.encrypt(sk_alice_fhe)
    C = C1 + C2
    sk_fhe = private_key.decrypt(C)
    PK_fhe_bob = get_PK(sk_fhe)
    HPK_fhe_bob = sha3.keccak_256(PK_fhe_bob[0].to_bytes(32,'big')+PK_fhe_bob[1].to_bytes(32,'big')).hexdigest()
    WA_fhe_bob = "0x"+ HPK_fhe_bob[-40:]
    end = time.perf_counter()
    elapsed_time = end - start
    print(elapsed_time)
    i+1


0.45094075900010466
0.6529816150000443
0.2070206700000199
0.45105119699996976
0.22050437899997632
0.40666856399980134
0.5448383209998156
0.1277644940000755
0.42657810600007906
0.7091162740000527
0.22314187200004199
0.4865496379998149
0.6904741609998837
0.46729610500005947
0.36159669599987865
0.14296112800002447
0.10880430799988972
0.31328044399992905
0.20744476300001224
0.4714497140000731
0.1791791570001351
0.22216753099996822
0.43617693999999574
0.4837115000000267
0.5941410230000201
0.5181789980001668
0.4543226460000369
0.4032761020000635
0.23474989800001822
0.4559833289999915
0.953031651000174
0.29404562400009127
0.9124160789999678
0.46758275699994556
0.45083524000006037
1.2330560030000015
0.5721667170000728
0.5178396479998355
0.32641611700000794
0.24978579000003265
0.1956587170000148
0.42525035799985744
0.2806287550001798
0.44257594399982736
0.3359002469999268
0.8760950930000035
0.2084497820001161
0.3117251150001721
1.9069394880000345
0.2701593760000378
0.3915598600001431
0.19526527

In [30]:

!pip install py_ecc
!pip uninstall sha3
!pip install ecdsa
!pip install pysha3
!pip install eclib



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [31]:
import hashlib
from py_ecc.secp256k1 import *
import sha3
from eth_account import Account
from ecdsa import SigningKey, SECP256k1 
from eclib import regev, gsw
from eclib.numutils import *
from eclib.randutils import *
from eclib.primeutils import *
from eclib.modutils import *
from collections import namedtuple
from math import ceil, log2
import numpy as np

In [32]:
def sk_PK_HPK_WA(sk):
    # PK = secp256k1.privtopub(s.to_bytes(32, "big"))
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))  
    HPK = sha3.keccak_256(PK[0].to_bytes(32, "big")+PK[1].to_bytes(32, "big")).hexdigest()
    WA = "0x"+ HPK[-40:]
    return PK, HPK, WA
    
def get_PK(sk):
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))
    return PK

In [33]:
for i in range(100):
    sum = 0
    start = time.perf_counter()
    sk_bob = int(0x13803677559c049a98a1576ad2f1fd15bdd9f81886f225c0850e883bd7d4cf63)
    PK_bob, HPK_bob, WA_bob = sk_PK_HPK_WA(sk_bob)
    key_length = 20
    params, PK_bob_fhe, sk_bob_fhe = keygen(key_length)
    sk_alice_fhe = int(0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30)
    C2 = encrypt(params, PK_bob_fhe, sk_bob)
    sk_bob_new = decrypt(params, sk_bob_fhe, C2)
    sk_alice_fhe = int(0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30)
    PK_alice_fhe = get_PK(sk_alice_fhe)
    PK_fhe_alice = secp256k1.add(PK_alice_fhe, PK_bob)
    HPK_fhe_alice = sha3.keccak_256(PK_fhe_alice[0].to_bytes(32,'big')+PK_fhe_alice[1].to_bytes(32,'big')).hexdigest()
    WA_fhe_alice = "0x"+ HPK_fhe_alice[-40:]
    C1 = encrypt(params, PK_bob_fhe, sk_alice_fhe)
    C = C1 + C2
    sk_fhe = decrypt(params, sk_bob_fhe, C)
    end = time.perf_counter()
    elapsed_time = end - start
    print(elapsed_time)
    i+1

NameError: name 'keygen' is not defined